In [186]:

from pymongo import MongoClient
import pandas as pd

client = MongoClient("mongodb://localhost:27017/")
database = client["Research"]
col_var = database["Variants"]


In [187]:
chrom_list = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,"X","Y"]

singleton_all = []

In [188]:
for chr in chrom_list: 
    
    pipeline= [
     {
        u"$match": {
            u"AC": 1.0,
            u"FILTER": u"PASS",
            u"CHROM": chr
        }
    }, 
    {
        u"$project": {
            u"AN": 1.0,
            u"vep_dictionary": 1.0,
            u"FILTER": 1.0,
            u"POS": 1.0,
            u"AN_male": 1.0,
            u"AN_female": 1.0,
            u"AC": 1.0,
            u"AC_male": 1.0,
            u"AC_female": 1.0
        }
    }, 
    {
        u"$lookup": {
            u"from": u"Coverage",
            u"localField": u"POS",
            u"foreignField": u"pos",
            u"as": u"med_coverage"
        }
    }, 
    {
        u"$unwind": {
            u"path": u"$med_coverage"
        }
    }, 
    {
        u"$match": {
            u"med_coverage.median": {
                u"$gte": 1.0
            },
            u"med_coverage.chrom": chr
        }
    }, 
    {
        u"$group": {
            u"_id": {
                u"Gene": u"$vep_dictionary.Feature",
                u"Consequence": u"$vep_dictionary.Consequence"
            },
            u"sum_AN_male": {
                u"$sum": u"$AN_male"
            },
            u"sum_AN_female": {
                u"$sum": u"$AN_female"
            },
            u"sum_AC_female": {
                u"$sum": u"$AC_female"
            },
            u"sum_AC_male": {
                u"$sum": u"$AC_male"
            },
            u"sum_AC": {
                u"$sum": u"$AC"
            },
            u"count": {
                u"$sum": 1.0
            }
        }
    }
    ]
    
    result = col_var.aggregate(pipeline)
    #list(result)
    result_df= pd.DataFrame(data = result)
    result_df['CHROM'] = str(chr)
    singleton_all.append(result_df)



In [190]:
all_chrom = pd.concat(singleton_all)


In [192]:
all_chrom= all_chrom[all_chrom._id != {}]


242326
242325


In [193]:
gene= []
Consequence= []
for i in range(len(all_chrom.iloc[:,0])):
    gene.append(all_chrom.iloc[i,0]['Gene'])
    Consequence.append(all_chrom.iloc[i,0]['Consequence'])

all_chrom['Gene']= gene
all_chrom['Consequence']= Consequence


In [194]:
cols = all_chrom.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]

all_chrom = all_chrom[cols]

In [195]:
all_chrom.head()

                Gene                           Consequence  \
0    ENST00000370500                        intron_variant   
1    ENST00000264126                   3_prime_UTR_variant   
2    ENST00000295566  splice_region_variant&intron_variant   
3    ENST00000344843  splice_region_variant&intron_variant   
4    ENST00000355356                    synonymous_variant   
..               ...                                   ...   
187  ENST00000338981                  splice_donor_variant   
188  ENST00000317961                    frameshift_variant   
189  ENST00000215479                    frameshift_variant   
190  ENST00000288666                    synonymous_variant   
191  ENST00000338981  splice_region_variant&intron_variant   

                                                   _id  sum_AN_male  \
0    {'Gene': 'ENST00000370500', 'Consequence': 'in...     10252506   
1    {'Gene': 'ENST00000264126', 'Consequence': '3_...      1068120   
2    {'Gene': 'ENST00000295566', 'Conseque

In [196]:
#all_chrom.drop(['_id'], axis=1)
del all_chrom['_id']

                Gene                           Consequence  sum_AN_male  \
0    ENST00000370500                        intron_variant     10252506   
1    ENST00000264126                   3_prime_UTR_variant      1068120   
2    ENST00000295566  splice_region_variant&intron_variant      3359190   
3    ENST00000344843  splice_region_variant&intron_variant      2138212   
4    ENST00000355356                    synonymous_variant      4955672   
..               ...                                   ...          ...   
187  ENST00000338981                  splice_donor_variant        56584   
188  ENST00000317961                    frameshift_variant       605191   
189  ENST00000215479                    frameshift_variant       135293   
190  ENST00000288666                    synonymous_variant      1011435   
191  ENST00000338981  splice_region_variant&intron_variant      1647610   

     sum_AN_female  sum_AC_female  sum_AC_male  sum_AC  count CHROM  
0          8654438           

In [197]:
import os
os.chdir("/home/ethel/Documents/all chrom")
all_chrom.to_csv("all_chr_singelton.csv")